In [1]:
import sys
import os

# Add the path to `database_scripts` folder
sys.path.append(os.path.abspath("../database_scripts"))
import  db_utils as du
import create_tables as ct


conn = ct.create_or_open_database()
df_fahrrad_Zähler = du.fetch_data_df('Standorten_Zählstelle',conn)
df_mess_Fahrrad= du.fetch_data_df('Messdaten_Fahrrad',conn)
df_bezirke= du.fetch_data_df('Bezirke',conn)


Establish SQLite connection.


In [ ]:
df_date = du.fetch_data_df('Date_dim',conn)
df_time = du.fetch_data_df('Time_dim',conn)
df_wetter = du.fetch_data_df('Wetter',conn)

In [6]:
#type conversion inside the dataframe
df_time['TimeID'] = df_time['TimeID'].astype('int')
df_date['DateID'] = df_date['DateID'].astype('int')
df_wetter['TimeID'] = df_wetter['TimeID'].astype('int')
df_wetter['DateID'] = df_wetter['DateID'].astype('int32')
df_date['year'] = df_date['year'].astype('int')
df_date['month'] = df_date['month'].astype('int')
df_date['day'] = df_date['day'].astype('int')
df_date['is_holiday'] = df_date['is_holiday'].astype('bool')
df_date['quarter'] = df_date['quarter'].astype('int')
df_mess_Fahrrad['TimeID'] = df_mess_Fahrrad['TimeID'].astype('int')
df_mess_Fahrrad['DateID'] = df_mess_Fahrrad['DateID'].astype('int')
#df_mess_Fahrrad['Wert'] = df_mess_Fahrrad['Wert'].astype('int32')

df_wetter['temperature_2m (°C)'] = df_wetter['temperature_2m (°C)'].astype('float32')
df_wetter['relative_humidity_2m (%)'] = df_wetter['relative_humidity_2m (%)'].astype('float32')
df_wetter['rain (mm)'] = df_wetter['rain (mm)'].astype('float32')
df_wetter['snowfall (cm)'] = df_wetter['snowfall (cm)'].astype('float32')
df_wetter['cloud_cover (%)'] = df_wetter['cloud_cover (%)'].astype('float32')


In [3]:
#df_na = df_mess_Fahrrad.isna()#.sum()
#df_mess_Fahrrad.fillna(0)

In [4]:
#df_na_rows = df_mess_Fahrrad[df_mess_Fahrrad.isna().any(axis=1)]
#print(df_na_rows)

In [5]:
#df_time['time_of_the_day'].unique()

In [7]:
#Feature Engineering:
#add a new colum for Zählstelle to be converted to number
df_fahrrad_Zähler['Zählstelle_ID'] = range(0, df_fahrrad_Zähler.shape[0])
df_fahrrad_Zähler['Zählstelle_ID'] = df_fahrrad_Zähler['Zählstelle_ID'].astype('int')
df_bezirke['Bezirk_ID'] =  range(0, df_bezirke.shape[0])
df_bezirke['Bezirk_ID'] = df_bezirke['Bezirk_ID'].astype('int')

def time_of_the_day_as_number(time_of_the_day):
    if time_of_the_day == 'nachts': 
        return 0
    if time_of_the_day == 'morgens':
        return 1
    if time_of_the_day == 'vormittags':
        return 2
    if time_of_the_day == 'mittags':
        return 3
    if time_of_the_day == 'nachmittags':
        return 4
    else:
        return 5
df_time['timeoftheday']=df_time['time_of_the_day'].apply(time_of_the_day_as_number)
df_time['timeoftheday'] = df_time['timeoftheday'].astype('int')

In [ ]:
df_merged = df_fahrrad_Zähler.merge(df_bezirke, on='Bezirk', how='left')
df_merged = df_merged.merge(df_mess_Fahrrad, on='Zählstelle', how='left')
df_merged = df_merged.rename(columns={'DateID': 'Date', 'TimeID':'Time'})
df_merged.to_csv('MessDatenFahrrad.csv')
#print(df_merged.info())


,Zählstelle,Bezirk,Beschreibung,Installationsdatum,Breitengrad,Längengrad,Geometry,DateID,TimeID,Wert
0,12-PA-SCH,Pankow,Schwedter Steg,2012-03-01 00:00:00,6.908842e+06,1.496162e+06,"MULTIPOLYGON (((13.388380799999998 52.5676794,...",01.01.2018,0,14
1,12-PA-SCH,Pankow,Schwedter Steg,2012-03-01 00:00:00,6.908842e+06,1.496162e+06,"MULTIPOLYGON (((13.388380799999998 52.5676794,...",01.01.2018,1,16
2,12-PA-SCH,Pankow,Schwedter Steg,2012-03-01 00:00:00,6.908842e+06,1.496162e+06,"MULTIPOLYGON (((13.388380799999998 52.5676794,...",01.01.2018,2,7
3,12-PA-SCH,Pankow,Schwedter Steg,2012-03-01 00:00:00,6.908842e+06,1.496162e+06,"MULTIPOLYGON (((13.388380799999998 52.5676794,...",01.01.2018,3,13
4,12-PA-SCH,Pankow,Schwedter Steg,2012-03-01 00:00:00,6.908842e+06,1.496162e+06,"MULTIPOLYGON (((13.388380799999998 52.5676794,...",01.01.2018,4,1


In [4]:
df_merged.size

14139600

In [ ]:
df_date = du.fetch_data_df('Date_dim',conn)
df_time = du.fetch_data_df('Time_dim',conn)
df_wetter = du.fetch_data_df('Wetter',conn)

In [ ]:
df_merged.drop(['time_of_the_day','Breitengrad', 'Längengrad', 'Geometry','Bezirk','Date','Beschreibung','Installationsdatum','Zählstelle'], axis = 1, inplace= True)

In [ ]:
#sns.jointplot(x = 'Time on App', y = 'Length of Membership', kind = 'hex', data = kunden)

In [ ]:
#Linear Regression ohne Anpassungen /Normalisierung 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import numpy as np
df_merged.dropna(inplace=True, axis= 0)  # the Bezirk Reinickendorf   wurde gelöscht  id= 7
X = df_merged.drop(['Wert'], axis = 1)
y = df_merged['Wert']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)
lr = LinearRegression()
lr.fit(X_train, y_train)
predictions = lr.predict(X_test)

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
import matplotlib.pyplot as plt
plt.scatter(y_test, predictions)
plt.xlabel('Korrekte Labels: y_test')
plt.ylabel('Vorhergesagten Labels: predictions')
# there is no real linera relation between the parameter and the target Wert (Anzahl) deswegen wir möchten mit einem andere Algorithm es probieren

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

df_merged.dropna(inplace=True, axis= 0)  # the Bezirk Reinickendorf   wurde gelöscht  id= 7
X = df_merged.drop(['Wert'], axis = 1)
y = df_merged['Wert']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=101)
scalar = StandardScaler()
X_test = scalar.fit_transform(X_test)
X_train = scalar.fit_transform(X_train)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor
knn_model = KNeighborsRegressor(n_neighbors=5)  # You can tune k as a hyperparameter
knn_model.fit(X_train, y_train)

# Step 4: Make predictions
y_pred = knn_model.predict(X_test)

# Step 5: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R²: {r2}')

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, predictions))
print(classification_report(y_test, predictions))

In [ ]:
from sklearn.svm import SVC
svr_model = SVC(kernel='rbf')
svr_model.fit(X_train, y_train)
y_pred = svr_model.predict(X_test)
# Step 5: Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f'MAE: {mae}')
print(f'MSE: {mse}')
print(f'R2: {r2}')